# HW3
### 根據UCI-Sonar資料集(2 classes, 60 features) 
1. 使用Single layer perceptron做聲納資料集分類
2. 和HW1的結果作比較

## 基本概念

### Neural Networks

是一個模仿生物神經網路的結構和功能的數學模型，用於對函式進行估計或近似。
* 具有一組可以被學習演算法調節的權重
* 可以估計輸入資料的非線性函式關係(例如:XOR)

#### Models of a Neuron
![](https://i.imgur.com/CZ31LGG.png)
$v_k = \sum_{j=1}^{p}w_{kj}x_{j}$, $y_k = \varphi(v_k-\theta_k)$

每個神經元結構如圖，通常具有多個輸入$x_1$~$ x_p$，$x_0$為偏差值，$y_k$則是透過激勵函式產生出來的output結果，並且成為下一輪的input。

當神經元某個輸入權重為0時，表示此神經元激發與否和這個輸入值無關，類神經網路會透過權重的更新，而達到學習的效果。
$$w_{kj}(n+1) = w_{kj}(n) + \Delta w_{kj}(n)$$
$$e_k(n) = d_k(n) - y_k(n), d_k(n)為期望輸出值$$
$$\Delta w_{kj}(n) = \eta e_k(n)x_j(n)$$
權重調整方式如下：
* 當第n筆訓練資料x(n)在第n輪時使用權重w(n)做計算，並得到正確的分類結果時，則

  $w(n+1) = w(n)$,  if $w^T(n)x(n) \geq 0$  and $x(n)$  belongs to class 1

  $w(n+1) = w(n)$,  if $w^T(n)x(n) < 0$ and $x(n)$ belongs to class 2
  
* 反之，則更新權重

  $w(n+1) = w(n) - \eta(n)x(n)$,  if $w^T(n)x(n) \geq 0$  and $x(n)$  belongs to class 2

  $w(n+1) = w(n) + \eta(n)x(n)$,  if $w^T(n)x(n) < 0$ and $x(n)$ belongs to class 1

### Summary of the Perceptron Convergence Algorithm

$x(n) = [-1, x_1(n), x_2(n), ..., x_p(n)]^T$: input vector

$w(n) = [ \theta(n), w_1(n), w_2(n), ..., w_p(n)]^T$: weight vector

$\theta(n)$: threshold

$y(n)$: actual response

$d(n)$: desired response

$\eta$: learning-rate parameter


* Step 1: 初始化權重，先任意設定權值，之後會透過更新而自動調整。
* Step 2: 輸入訓練樣本，和期待值$d(n)$，計算加權總值
* Step 3: 透過激勵函數產生 $y(n)=sgn[w^T(n)x(n)]$
* Step 4: 根據結果更新權重$w(n+1)=w(n)+\eta[d(n)-y(n)]x(n)$

  $d(n)=+1$ if $x(n)$ belongs to class 1, $d(n)=-1$ if $x(n)$ belongs to class 2
* Step 5: 回到step 2，進行下一輪訓練

### 演算法缺點

1. 一定要線性可分，演算法才會停止，否則要另設停下的機制，例如在此作業中我們設定跑500次後停下來，或者可以設定當錯誤小於某一數值後停下。
2. 只能知道分類結果式A類或B類，但無法知道機率多少。

## 資料集說明

### Connectionist Bench (Sonar, Mines vs. Rocks) Data Set
 dataset: http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)
 * Number of Instances: 208
 * Number of Attributes: 60
 
information:
 * sonar.mines: contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions.
 * sonar.rocks: contains 97 patterns obtained from rocks under similar conditions.
 * The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.

## 實作

### 連接Google Drive

In [165]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls '/content/drive/My Drive/Pattern_Recognition/HW3'

Mounted at /content/drive
HW3.ipynb  sonar.all-data


### 讀取資料
Rock為1, Mine為0

In [220]:
file = '/content/drive/My Drive/Pattern_Recognition/HW3/sonar.all-data'
data = []
with open(file,'r') as f:
  for line in f:
    features = line.strip('\n').split(',')
    features.insert(0, -1)
    data.append(features)

print('number of data:', len(data))
print(data[0])
print(data[-1])

number of data: 208
[-1, '0.0200', '0.0371', '0.0428', '0.0207', '0.0954', '0.0986', '0.1539', '0.1601', '0.3109', '0.2111', '0.1609', '0.1582', '0.2238', '0.0645', '0.0660', '0.2273', '0.3100', '0.2999', '0.5078', '0.4797', '0.5783', '0.5071', '0.4328', '0.5550', '0.6711', '0.6415', '0.7104', '0.8080', '0.6791', '0.3857', '0.1307', '0.2604', '0.5121', '0.7547', '0.8537', '0.8507', '0.6692', '0.6097', '0.4943', '0.2744', '0.0510', '0.2834', '0.2825', '0.4256', '0.2641', '0.1386', '0.1051', '0.1343', '0.0383', '0.0324', '0.0232', '0.0027', '0.0065', '0.0159', '0.0072', '0.0167', '0.0180', '0.0084', '0.0090', '0.0032', 'R']
[-1, '0.0260', '0.0363', '0.0136', '0.0272', '0.0214', '0.0338', '0.0655', '0.1400', '0.1843', '0.2354', '0.2720', '0.2442', '0.1665', '0.0336', '0.1302', '0.1708', '0.2177', '0.3175', '0.3714', '0.4552', '0.5700', '0.7397', '0.8062', '0.8837', '0.9432', '1.0000', '0.9375', '0.7603', '0.7123', '0.8358', '0.7622', '0.4567', '0.1715', '0.1549', '0.1641', '0.1869', '0.26

### 在資料集中選擇50%資料作為training data, 50%為test data

In [0]:
# 隨機分配資料
import random
print('before shuffle:')
for i in data:
  print(i[-1], end=' ')
print()
random.shuffle(data)
print('after shuffle:')
for i in data:
  print(i[-1], end=' ')
print()

train = data[0: int(len(data)/2)]
test = data[int(len(data)/2): int(len(data))]
train_label = []
test_label = []

for i in train:
  c = i.pop()
  if(c == 'R'):
    train_label.append(1)
  elif(c == 'M'):
    train_label.append(0)
for i in test:
  c = i.pop()
  if(c == 'R'):
    test_label.append(1)
  elif(c == 'M'):
    test_label.append(0)

print('number of training set:', len(train))
print('number of testing set:', len(test))
print('number of training label:', len(train_label))
print('number of testing label:', len(test_label))

In [221]:
# 平均分配
import random
train = []
test = []
for i in range(len(data)):
  if i % 2 == 1:
    train.append(data[i])
  else:
    test.append(data[i])

random.shuffle(train)
random.shuffle(test)

train_label = []
test_label = []

for i in train:
  c = i.pop()
  if(c == 'R'):
    train_label.append(1)
  elif(c == 'M'):
    train_label.append(0)
for i in test:
  c = i.pop()
  if(c == 'R'):
    test_label.append(1)
  elif(c == 'M'):
    test_label.append(0)

print('number of training set:', len(train))
print('number of testing set:', len(test))
print('number of training label:', len(train_label))
print('number of testing label:', len(test_label))

number of training set: 104
number of testing set: 104
number of training label: 104
number of testing label: 104


### Perceptron Convergence Algorithm

In [0]:
import numpy as np
import random

# 訓練次數
num = 500
# 學習速率
learning = 0.1
# 權重
w = []
for i in range(len(train[0])):
  w.append(random.uniform(0, 1))
train = np.array(train).astype(np.float)
test = np.array(test).astype(np.float)
w = np.array(w).astype(np.float)

In [0]:
for i in range(num):
  for j in range(len(train)):
    y = 1 if train[j]@w >= 0 else 0
    w = w + learning * (train_label[j] - y) * train[j]

In [224]:
# 計算錯誤率
error = 0
for i in range(len(test)):
  y = 1 if test[i]@w >= 0 else 0
  if test_label[i] != y:
    error += 1
    
print('error rate:', error/len(test)*100, '%')

error rate: 25.961538461538463 %


In [225]:
# 計算錯誤率
error = 0
for i in range(len(train)):
  y = 1 if train[i]@w >= 0 else 0
  if train_label[i] != y:
    error += 1
    
print('error rate:', error/len(test)*100, '%')

error rate: 20.192307692307693 %


## 參考資料
* [Pattern Recognition課堂講義](https://drphototw.wixsite.com/wujl/teaching)
* [人工神經網路](https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)
* [反向傳播算法](https://zh.wikipedia.org/wiki/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD%E7%AE%97%E6%B3%95)
